In [11]:
def rbf(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)

In [ ]:
def kmeans(X, k):
    """Performs k-means clustering for 1D input
    
    Arguments:
        X {ndarray} -- A Mx1 array of inputs
        k {int} -- Number of clusters
    
    Returns:
        ndarray -- A kx1 array of final cluster centers
    """
    # randomly select initial clusters from input data
    clusters = np.random.choice(np.squeeze(X), size=k)
    prevClusters = clusters.copy()
    stds = np.zeros(k)
    converged = False
    while not converged:
        """
        compute distances for each cluster center to each point 
        where (distances[i, j] represents the distance between the ith point and jth cluster)
        """
        distances = np.squeeze(np.abs(X[:, np.newaxis] - clusters[np.newaxis, :]))
        # find the cluster that's closest to each point
        closestCluster = np.argmin(distances, axis=1)
        # update clusters by taking the mean of all of the points assigned to that cluster
        for i in range(k):
            pointsForCluster = X[closestCluster == i]
            if len(pointsForCluster) > 0:
                clusters[i] = np.mean(pointsForCluster, axis=0)
        # converge if clusters haven't moved
        converged = np.linalg.norm(clusters - prevClusters) < 1e-6
        prevClusters = clusters.copy()
    distances = np.squeeze(np.abs(X[:, np.newaxis] - clusters[np.newaxis, :]))
    closestCluster = np.argmin(distances, axis=1)
    clustersWithNoPoints = []
    for i in range(k):
        pointsForCluster = X[closestCluster == i]
        if len(pointsForCluster) < 2:
            # keep track of clusters with no points or 1 point
            clustersWithNoPoints.append(i)
            continue
        else:
            stds[i] = np.std(X[closestCluster == i])
    # if there are clusters with 0 or 1 points, take the mean std of the other clusters
    if len(clustersWithNoPoints) > 0:
        pointsToAverage = []
        for i in range(k):
            if i not in clustersWithNoPoints:
                pointsToAverage.append(X[closestCluster == i])
        pointsToAverage = np.concatenate(pointsToAverage).ravel()
        stds[clustersWithNoPoints] = np.mean(np.std(pointsToAverage))
    return clusters, std

In [13]:
class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""
    def __init__(self, k=2, lr=0.01, epochs=100, rbf=rbf, inferStds=True):
        self.k = k
        self.lr = lr
        self.epochs = epochs
        self.rbf = rbf
        self.inferStds = inferStds
        self.w = np.random.randn(k)
        self.b = np.random.randn(1)
        
    def fit(self, X, y):
        if self.inferStds:
        # compute stds from data
            self.centers, self.stds = kmeans(X, self.k)
        else:
        # use a fixed std 
            self.centers, _ = kmeans(X, self.k)
            dMax = max([np.abs(c1 - c2) for c1 in self.centers for c2 in self.centers])
            self.stds = np.repeat(dMax / np.sqrt(2*self.k), self.k)
    # training
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
            # forward pass
                a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                F = a.T.dot(self.w) + self.b
                loss = (y[i] - F).flatten() ** 2
                print('Loss: {0:.2f}'.format(loss[0]))
                # backward pass
                error = -(y[i] - F).flatten()
                # online update
                self.w = self.w - self.lr * a * error
                self.b = self.b - self.lr * error
                
    def predict(self, X):
        y_pred = []
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w) + self.b
            y_pred.append(F)
        return np.array(y_pred)

In [17]:
def Func(a, b, x):
    return (a * (b ** x))

In [40]:
%matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.special
from numpy.random import uniform

a = 2.4
b = 4.2

X = np.random.uniform(0., 1., 100)
X = np.sort(X, axis=0)
noise = np.random.uniform(-0.1, 0.1, 100)
y = Func(a, b, X) + noise

rbfnet = RBFNet(lr=0.01, k=30)
rbfnet.fit(X, y)
y_pred = rbfnet.predict(X)
plt.plot(X, y, '-o', label='true')
plt.plot(X, y_pred, '-o', label='RBF-Net')
plt.legend()
plt.tight_layout()
plt.show()

Using matplotlib backend: Qt5Agg
Loss: 13.01
Loss: 12.64
Loss: 11.19
Loss: 10.73
Loss: 10.39
Loss: 9.05
Loss: 11.34
Loss: 9.83
Loss: 9.02
Loss: 10.33
Loss: 7.99
Loss: 5.59
Loss: 5.19
Loss: 6.68
Loss: 6.35
Loss: 5.34
Loss: 4.62
Loss: 5.09
Loss: 6.47
Loss: 5.63
Loss: 5.42
Loss: 4.54
Loss: 4.21
Loss: 5.29
Loss: 4.61
Loss: 4.86
Loss: 7.84
Loss: 7.22
Loss: 5.42
Loss: 3.26
Loss: 1.60
Loss: 1.69
Loss: 3.06
Loss: 2.81
Loss: 2.32
Loss: 3.03
Loss: 2.76
Loss: 3.58
Loss: 2.93
Loss: 2.29
Loss: 3.31
Loss: 4.05
Loss: 3.85
Loss: 2.66
Loss: 4.25
Loss: 3.73
Loss: 4.20
Loss: 3.97
Loss: 4.57
Loss: 4.29
Loss: 4.99
Loss: 4.46
Loss: 3.87
Loss: 4.33
Loss: 3.88
Loss: 3.81
Loss: 3.61
Loss: 3.87
Loss: 5.35
Loss: 5.00
Loss: 5.07
Loss: 6.03
Loss: 7.20
Loss: 10.92
Loss: 10.74
Loss: 16.16
Loss: 20.69
Loss: 15.20
Loss: 22.72
Loss: 24.09
Loss: 26.10
Loss: 25.41
Loss: 14.63
Loss: 12.47
Loss: 9.70
Loss: 12.07
Loss: 23.12
Loss: 21.33
Loss: 22.49
Loss: 22.76
Loss: 22.38
Loss: 24.03
Loss: 20.89
Loss: 19.82
Loss: 25.27
Loss

Loss: 0.76
Loss: 1.78
Loss: 2.04
Loss: 2.58
Loss: 2.55
Loss: 0.61
Loss: 0.11
Loss: 0.12
Loss: 0.01
Loss: 1.09
Loss: 0.57
Loss: 0.54
Loss: 0.61
Loss: 0.76
Loss: 1.54
Loss: 0.47
Loss: 0.37
Loss: 1.78
Loss: 2.58
Loss: 2.06
Loss: 3.86
Loss: 3.34
Loss: 2.55
Loss: 2.14
Loss: 2.20
Loss: 3.87
Loss: 4.51
Loss: 5.08
Loss: 5.05
Loss: 7.44
Loss: 9.20
Loss: 8.39
Loss: 8.59
Loss: 5.18
Loss: 3.02
Loss: 1.98
Loss: 1.64
Loss: 1.26
Loss: 1.38
Loss: 1.43
Loss: 1.98
Loss: 1.70
Loss: 1.02
Loss: 0.98
Loss: 1.16
Loss: 0.89
Loss: 0.57
Loss: 0.44
Loss: 0.36
Loss: 0.33
Loss: 0.19
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.02
Loss: 0.05
Loss: 0.04
Loss: 0.06
Loss: 0.73
Loss: 0.72
Loss: 0.37
Loss: 0.05
Loss: 0.06
Loss: 0.05
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.03
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.02
Loss: 0.00
Loss: 0.01
Loss: 0.03
Loss: 0.04
Loss: 0.13
Loss: 0.10
Loss: 0.20
Loss: 0.12

Loss: 0.02
Loss: 0.06
Loss: 0.17
Loss: 0.34
Loss: 0.25
Loss: 0.08
Loss: 0.04
Loss: 0.07
Loss: 0.03
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.15
Loss: 0.13
Loss: 0.20
Loss: 0.11
Loss: 0.08
Loss: 0.35
Loss: 0.29
Loss: 0.29
Loss: 0.95
Loss: 0.92
Loss: 0.52
Loss: 0.16
Loss: 0.00
Loss: 0.00
Loss: 0.03
Loss: 0.04
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.04
Loss: 0.01
Loss: 0.00
Loss: 0.02
Loss: 0.00
Loss: 0.01
Loss: 0.04
Loss: 0.03
Loss: 0.08
Loss: 0.05
Loss: 0.04
Loss: 0.07
Loss: 0.13
Loss: 0.04
Loss: 0.13
Loss: 0.18
Loss: 0.20
Loss: 0.32
Loss: 0.25
Loss: 0.40
Loss: 0.34
Loss: 0.17
Loss: 0.35
Loss: 0.45
Loss: 0.32
Loss: 0.17
Loss: 0.25
Loss: 0.22
Loss: 0.00
Loss: 0.16
Loss: 0.00
Loss: 0.11
Loss: 0.16
Loss: 0.32
Loss: 0.33
Loss: 0.00
Loss: 0.25
Loss: 1.36
Loss: 0.59
Loss: 0.00
Loss: 0.08
Loss: 0.09
Loss: 0.07
Loss: 0.03
Loss: 0.05
Loss: 0.15
Loss: 0.20
Loss: 0.08
Loss: 0.17
Loss: 0.06
Loss: 0.65
Loss: 0.38
Loss: 0.07
Loss: 0.01
Loss: 0.02
Loss: 0.60
Loss: 0.97
Loss: 1.04

Loss: 0.26
Loss: 0.18
Loss: 0.36
Loss: 0.41
Loss: 0.29
Loss: 0.20
Loss: 0.34
Loss: 0.31
Loss: 0.01
Loss: 0.07
Loss: 0.01
Loss: 0.02
Loss: 0.05
Loss: 0.13
Loss: 0.14
Loss: 0.03
Loss: 0.31
Loss: 1.36
Loss: 0.65
Loss: 0.01
Loss: 0.14
Loss: 0.15
Loss: 0.13
Loss: 0.06
Loss: 0.01
Loss: 0.25
Loss: 0.31
Loss: 0.02
Loss: 0.05
Loss: 0.00
Loss: 0.39
Loss: 0.18
Loss: 0.00
Loss: 0.02
Loss: 0.01
Loss: 0.35
Loss: 0.65
Loss: 0.59
Loss: 0.47
Loss: 1.97
Loss: 2.62
Loss: 1.34
Loss: 1.89
Loss: 2.28
Loss: 0.48
Loss: 0.02
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.10
Loss: 0.22
Loss: 0.15
Loss: 0.04
Loss: 0.01
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.02
Loss: 0.17
Loss: 0.12
Loss: 0.19
Loss: 0.13
Loss: 0.13
Loss: 0.39
Loss: 0.32
Loss: 0.31
Loss: 0.71
Loss: 0.69
Loss: 0.38
Loss: 0.13
Loss: 0.02
Loss: 0.01
Loss: 0.05
Loss: 0.06
Loss: 0.01
Loss: 0.02
Loss: 0.01
Loss: 0.05
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.03
Loss: 0.03
Loss: 0.07
Loss: 0.05
Loss: 0.04

Loss: 0.28
Loss: 1.10
Loss: 0.53
Loss: 0.01
Loss: 0.13
Loss: 0.12
Loss: 0.10
Loss: 0.05
Loss: 0.01
Loss: 0.24
Loss: 0.29
Loss: 0.02
Loss: 0.02
Loss: 0.00
Loss: 0.30
Loss: 0.13
Loss: 0.00
Loss: 0.05
Loss: 0.04
Loss: 0.27
Loss: 0.55
Loss: 0.38
Loss: 0.25
Loss: 1.69
Loss: 2.20
Loss: 0.82
Loss: 1.39
Loss: 2.33
Loss: 0.44
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.10
Loss: 0.22
Loss: 0.15
Loss: 0.05
Loss: 0.01
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.02
Loss: 0.13
Loss: 0.07
Loss: 0.12
Loss: 0.11
Loss: 0.14
Loss: 0.35
Loss: 0.29
Loss: 0.28
Loss: 0.46
Loss: 0.46
Loss: 0.23
Loss: 0.09
Loss: 0.03
Loss: 0.03
Loss: 0.06
Loss: 0.06
Loss: 0.01
Loss: 0.03
Loss: 0.01
Loss: 0.05
Loss: 0.02
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.02
Loss: 0.02
Loss: 0.02
Loss: 0.05
Loss: 0.03
Loss: 0.02
Loss: 0.04
Loss: 0.06
Loss: 0.00
Loss: 0.07
Loss: 0.13
Loss: 0.10
Loss: 0.11
Loss: 0.07
Loss: 0.12
Loss: 0.15
Loss: 0.14
Loss: 0.29
Loss: 0.27
Loss: 0.18
Loss: 0.17
Loss: 0.33

...

Loss: 0.02
Loss: 0.02
Loss: 0.00
Loss: 0.04
Loss: 0.08
Loss: 0.04
Loss: 0.03
Loss: 0.01
Loss: 0.03
Loss: 0.06
Loss: 0.09
Loss: 0.20
Loss: 0.13
Loss: 0.07
Loss: 0.12
Loss: 0.28
Loss: 0.25
Loss: 0.03
Loss: 0.03
Loss: 0.03
Loss: 0.00
Loss: 0.00
Loss: 0.03
Loss: 0.04
Loss: 0.06
Loss: 0.19
Loss: 0.68
Loss: 0.32
Loss: 0.00
Loss: 0.08
Loss: 0.06
Loss: 0.05
Loss: 0.02
Loss: 0.02
Loss: 0.18
Loss: 0.22
Loss: 0.02
Loss: 0.00
Loss: 0.01
Loss: 0.25
Loss: 0.09
Loss: 0.01
Loss: 0.09
Loss: 0.06
Loss: 0.22
Loss: 0.48
Loss: 0.19
Loss: 0.08
Loss: 1.42
Loss: 1.75
Loss: 0.31
Loss: 0.86
Loss: 2.38
Loss: 0.39
Loss: 0.00
Loss: 0.01
Loss: 0.03
Loss: 0.00
Loss: 0.11
Loss: 0.21
Loss: 0.14
Loss: 0.07
Loss: 0.02
Loss: 0.02
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.07
Loss: 0.02
Loss: 0.05
Loss: 0.08
Loss: 0.13
Loss: 0.27
Loss: 0.21
Loss: 0.21
Loss: 0.19
Loss: 0.20
Loss: 0.07
Loss: 0.04
Loss: 0.06
Loss: 0.05
Loss: 0.06
Loss: 0.06
Loss: 0.02
Loss: 0.04
Loss: 0.01
Loss: 0.04
Loss: 0.02

Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.05
Loss: 0.01
Loss: 0.03
Loss: 0.07
Loss: 0.13
Loss: 0.24
Loss: 0.19
Loss: 0.20
Loss: 0.13
Loss: 0.14
Loss: 0.04
Loss: 0.03
Loss: 0.06
Loss: 0.05
Loss: 0.05
Loss: 0.05
Loss: 0.02
Loss: 0.04
Loss: 0.01
Loss: 0.04
Loss: 0.02
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.02
Loss: 0.01
Loss: 0.01
Loss: 0.01
Loss: 0.01
Loss: 0.01
Loss: 0.03
Loss: 0.07
Loss: 0.03
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.04
Loss: 0.08
Loss: 0.17
Loss: 0.08
Loss: 0.04
Loss: 0.11
Loss: 0.25
Loss: 0.23
Loss: 0.03
Loss: 0.02
Loss: 0.04
Loss: 0.00
Loss: 0.00
Loss: 0.03
Loss: 0.03
Loss: 0.06
Loss: 0.16
Loss: 0.54
Loss: 0.26
Loss: 0.00
Loss: 0.07
Loss: 0.04
Loss: 0.04
Loss: 0.02
Loss: 0.02
Loss: 0.15
Loss: 0.19
Loss: 0.02
Loss: 0.00
Loss: 0.02
Loss: 0.23
Loss: 0.08
Loss: 0.01
Loss: 0.09
Loss: 0.07
Loss: 0.21
Loss: 0.46
Loss: 0.13
Loss: 0.04
Loss: 1.32
Loss: 1.59
Loss: 0.18
Loss: 0.69
Loss: 2.37
Loss: 0.36
Loss: 0.00
Loss: 0.01

Loss: 0.13
Loss: 0.16
Loss: 0.03
Loss: 0.00
Loss: 0.02
Loss: 0.22
Loss: 0.08
Loss: 0.01
Loss: 0.10
Loss: 0.07
Loss: 0.20
Loss: 0.44
Loss: 0.09
Loss: 0.01
Loss: 1.23
Loss: 1.45
Loss: 0.09
Loss: 0.55
Loss: 2.35
Loss: 0.34
Loss: 0.01
Loss: 0.02
Loss: 0.05
Loss: 0.01
Loss: 0.11
Loss: 0.18
Loss: 0.12
Loss: 0.09
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.04
Loss: 0.00
Loss: 0.02
Loss: 0.06
Loss: 0.13
Loss: 0.20
Loss: 0.16
Loss: 0.18
Loss: 0.07
Loss: 0.09
Loss: 0.02
Loss: 0.02
Loss: 0.07
Loss: 0.06
Loss: 0.04
Loss: 0.04
Loss: 0.02
Loss: 0.04
Loss: 0.01
Loss: 0.03
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.01
Loss: 0.02
Loss: 0.02
Loss: 0.06
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.02
Loss: 0.07
Loss: 0.15
Loss: 0.05
Loss: 0.02
Loss: 0.09
Loss: 0.22
Loss: 0.20
Loss: 0.03
Loss: 0.02
Loss: 0.04
Loss: 0.00
Loss: 0.00
Loss: 0.02
Loss: 0.02
Loss: 0.07

Loss: 0.07
Loss: 0.02
Loss: 0.10
Loss: 0.08
Loss: 0.19
Loss: 0.43
Loss: 0.05
Loss: 0.00
Loss: 1.13
Loss: 1.27
Loss: 0.02
Loss: 0.39
Loss: 2.31
Loss: 0.31
Loss: 0.02
Loss: 0.03
Loss: 0.06
Loss: 0.01
Loss: 0.10
Loss: 0.14
Loss: 0.09
Loss: 0.09
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.03
Loss: 0.00
Loss: 0.01
Loss: 0.05
Loss: 0.13
Loss: 0.16
Loss: 0.13
Loss: 0.16
Loss: 0.03
Loss: 0.04
Loss: 0.00
Loss: 0.01
Loss: 0.08
Loss: 0.07
Loss: 0.03
Loss: 0.03
Loss: 0.02
Loss: 0.04
Loss: 0.00
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.02
Loss: 0.02
Loss: 0.05
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.06
Loss: 0.12
Loss: 0.02
Loss: 0.00
Loss: 0.08
Loss: 0.19
Loss: 0.17
Loss: 0.04
Loss: 0.02
Loss: 0.04
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.02
Loss: 0.07
Loss: 0.11
Loss: 0.28
Loss: 0.13
Loss: 0.00
Loss: 0.05
Loss: 0.02

Loss: 0.00
Loss: 0.01
Loss: 0.02
Loss: 0.08
Loss: 0.10
Loss: 0.23
Loss: 0.11
Loss: 0.00
Loss: 0.04
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.03
Loss: 0.09
Loss: 0.11
Loss: 0.03
Loss: 0.00
Loss: 0.03
Loss: 0.20
Loss: 0.07
Loss: 0.02
Loss: 0.10
Loss: 0.08
Loss: 0.19
Loss: 0.42
Loss: 0.03
Loss: 0.00
Loss: 1.08
Loss: 1.20
Loss: 0.01
Loss: 0.33
Loss: 2.28
Loss: 0.29
Loss: 0.02
Loss: 0.03
Loss: 0.07
Loss: 0.01
Loss: 0.10
Loss: 0.13
Loss: 0.08
Loss: 0.09
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.03
Loss: 0.00
Loss: 0.00
Loss: 0.04
Loss: 0.13
Loss: 0.14
Loss: 0.11
Loss: 0.15
Loss: 0.02
Loss: 0.03
Loss: 0.00
Loss: 0.01
Loss: 0.09
Loss: 0.07
Loss: 0.03
Loss: 0.03
Loss: 0.02
Loss: 0.04
Loss: 0.00
Loss: 0.02
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.01
Loss: 0.01
Loss: 0.00
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.03
Loss: 0.01
Loss: 0.05
Loss: 0.01
Loss: 0.00
Loss: 0.00
Loss: 0.00
Loss: 0.01
Loss: 0.06
Loss: 0.11